![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, I will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. I will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

For this analysis I've four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [10]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

## The initial steps
In the first section of this assignment, I'll answer the question: 'What is the volume of products and average revenue for Addias and Nike products based on listing price quartiles?'.
First, I'll merge the four datasets into one DataFrame. This is done three steps, the datasets are alle merged on the 'product_id' column.

In [11]:
# Merging of DataFrames
merged_df = brands.merge(finance, on='product_id')
merged_df = merged_df.merge(info, on='product_id')
merged_df = merged_df.merge(reviews, on='product_id')
merged_df.dropna(inplace=True)

# Showing the result of the merged DataFrames
merged_df.head()

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0


## Starting the analysis
In the table above we can see a small section of the DataFrame I've just created.
Now I'll make the analysis to answer the earlier asked question. To do this I'm creating four quantiles to label the prices for each brand, as well as count the numbers of products in each quantile plus calculating the mean of the revenue.

In [12]:
# Creating the column price_label
merged_df['price_label'] = pd.qcut(merged_df['listing_price'], 4, labels=['Budget', 'Average', 'Expensive', 'Elite'])

# Creating the adidas_vs_nike DataFrame
adidas_vs_nike = merged_df.groupby(['brand', 'price_label'], as_index=False).agg(num_products=('price_label', 'count'), mean_revenue=('revenue', 'mean')).round(2)

# Showing the results
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


## First step of the analysis
Let's take a look at the first step of the analysis. Here we see that this particular web shop has more Adidas products than Nike. We can also see that the number of products seems to follow a normal distribution, for the Adidas products. Meaning that there are fewer products with the price labels 'Budget' and 'Elite' and more products in the categories 'Average' and 'Expensive'. While the situation is completely different for the Nike products.
Another difference between the two brands is how the 'mean_evenue' increases each time we go to a more expensive 'price_label' for the Adidas products, while for the Nike products we see the biggest 'mean_revenue' for the products with the Budget 'price_label', which also contains of more products, second comes the Elite product, which contains of second most products.
## Word count vs. mean rating
In this next section I'll analyze whether there is a difference between the word count of a product's description and its mean rating. This is done by splitting the product description length into bins and assign labels, before calculating the average rating and number of reviews for each range of description length.


In [13]:
# Creating a new column 'description_length'
merged_df['description_length'] = merged_df['description'].str.len()

# Creating a list of limits and a list of labels
limits = [0, 100, 200, 300, 400, 500, 600, 700]
label = ['100', '200', '300', '400', '500', '600', '700']

# Assigning the 'description_length' to bins
merged_df['description_length'] = pd.cut(merged_df['description_length'], bins=limits, labels=label)

# Comparing the mean rating with number of reviews grouped by 'description_length'
description_lengths = merged_df.groupby('description_length', as_index=False).agg(mean_rating=('rating', 'mean'), num_reviews=('reviews', 'count')).round(2)

# Showing the results
description_lengths

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15


## Rating vs Reviews
Let's have a look at the results for the comparison of 'description_length' and 'mean_rating'. The differences between the 'mean_rating' and 'description_length' are more or less the same for all the categories. The only outliers are the descriptions from 0-100 words, from 500-600 words and from 600-700 words. The first one is very low compared to the others, the second one is a bit lower than the others, and the last one is bit larger than the rest. The reason behind this distinction might be low number of reviews. The first one only has 7 reviews, the second and the third both have 15 reviews. So here a good or a bad reviews has a big influence. The results also show that the descriptions of length between 100–200-word, 200-300-words and 300-400-words gets by far the most amount of reviews. A simple explanation for this could be that there are more products that matches these description lengths.
## Revenue between clothing and footwear
In this last section I'll analyze how the volume of products and median revenue vary between clothing and footwear. To do so I'll search for words associated with footwear, then find clothing items by excluding products returned in the original search. Then I'll to build a DataFrame with the volume of products and median revenue for each type.

In [14]:
# List of footwear keywords
keywords = 'shoe*|trainer*|foot*'

# Creating footwear DataFrame
footwear = merged_df[merged_df['description'].str.contains(keywords)]

# Creating clothing DataFrame
clothing = merged_df[~merged_df.isin(footwear['product_id'])]
clothing.dropna(inplace=True)

# Creating the 'product_type' DataFrame
product_types = pd.DataFrame({'num_clothing_products': len(clothing),
                            'median_clothing_revenue': clothing['revenue'].median(),
                            'num_footwear_products': len(footwear),
                            'median_footwear_revenue': footwear['revenue'].median()},
                           index=[0])

# Showing the results
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,478,625.07,2639,3073.3


## Clothing or Footwear?
From the results of the analysis, we can see that there are 478 different pieces of clothing and 2639 different types of footwear. When we look at the median revenue for these two categories, then we get an idea of why there are a bigger revenue among the selection of footwear. Because the 'median_clothing_revenue' is only 625.07 in comparison to the 3073.3 for the 'median_footwear_renevue'.
## Where do we go from here
How could we potentially use this analysis to optimize ‘our’ web shop? For the first part of the analysis, where we ask the question: ‘What is the volume of products and average revenue for Addias and Nike products based on listing price quartiles?'. This part of the analysis opens for further investigation in the products in each ‘price_label’-category. For example, should it be possible to reduce the numbers of Adidas products, since there is a lot of products in each category, and not all products can sell equally good. We might consider taking in more Nike product, since we have very few compared to Adidas. Especially products in the Average-category, since it has a pretty good ‘mean_revenue’, when we consider it only contains of 8 different products.
For the second question, the one about if there is a difference between the word count of a product's description and its mean rating. This part of the analysis we can use to improve the description of our products. From what we know now we can't tell, if there is a correlation between how many products we're selling and the length of the product description. It is what the preliminary analysis is indicating, but we need to investigate that further on. 
The third and final question of this analysis was: 'How does the volume of products and median revenue vary between clothing and footwear?'. For further investigation regarding this question then we can for example use the analysis to look at the composition of the two types of products we are selling (clothing & footwear). For the footwear we can look the sells numbers for each product in this category and see if we can reduce the number of different shoes we're selling. For the clothing we can do the opposite and see if there any products in this category that we should have a bigger selection of.
